In [3]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten, Reshape, Conv2D, MaxPool2D,Conv2DTranspose

encoder = Sequential([Dense(2, input_shape=[3])])
decoder = Sequential([Dense(3, input_shape=[2])])

autoencoder = Sequential([encoder, decoder])

autoencoder.compile(loss="mse", optimizer = keras.optimizers.SGD(lr=0.1))

In [4]:
history = autoencoder.fit(X_train, X_train, epochs = 20)
codings = encoder.predict(X_train)

NameError: name 'X_train' is not defined

In [6]:
stacked_encoder = Sequential([
    Flatten(input_shape=[28,28]),
    Dense(100, activation ="selu"),
    Dense(30, activation ="selu")
])
stacked_decoder = Sequential([
    Dense(100, activation ="selu", input_shape=[30]),
    Dense(28*28, activation ="sigmoid"),
    Reshape([28,28])
])
stacked_ae = Sequential([stacked_encoder,stacked_decoder])
stacked_ae.compile(loss="binary_crossentropy", optimizer = keras.optimizer.SGD(lr=1.5))
history = stacked_ae.fit(X_train,X_train, epochs=10, validation_data=[X_valid,X_valid])

NameError: name 'Reshape' is not defined

In [2]:
from sklearn.manifold import TSNE

X_valid_compressed = stacked_encoder.predict(X_valid)
tsne = TSNE()
X_valid_2D = tsne.fit_transform(X_valid_compressed)

plt.scatter(X_valid_2D[:,0], X_valid_2D[:,1], c = y_valid, s=10, cmap="tab10")

NameError: name 'stackedencoder' is not defined

**Convolutional Autoencoders**

In [6]:
conv_encoder = Sequential([
    Reshape([28,28,1], input_shape = [28,28]),
    Conv2D(16, kernel_size=3, padding="same", activation="selu"),
    MaxPool2D(pool_size=2),
    Conv2D(32,kernel_size=3,padding="same",activation="selu"),
    MaxPool2D(pool_size=2),
    Conv2D(64,kernel_size=3,padding="same",activation="selu"),
    MaxPool2D(pool_size=2),
])
conv_decoder = Sequential([
    Conv2DTranspose(32, kernel_size = 3, strides=2, padding="valid", activation="selu", input_shape=[3,3,64]),
    Conv2DTranspose(16, kernel_size = 3, strides=2, padding="same", activation="selu"),
    Conv2DTranspose(1, kernel_size = 3, strides=2, padding="same", activation="sigmoid"),
    Reshape([28,28])
])
conv_ae = Sequential([conv_encoder, conv_decoder])

NameError: name 'Conv2DTranspose' is not defined

**Generative Adversarial Networks** (Fashion MNist)

In [7]:
codings_size = 30

generator = Sequential([
    Dense(100, activation ="selu", input_shape=[codings_size]),
    Dense(150, activation ="selu"),
    Dense(28*28, activation="sigmoid"),
    Reshape([28,28])
])

discriminator = Sequential([
    Flatten(input_shape=[28,28]),
    Dense(150, activation ="selu"),
    Dense(100, activation ="selu"),
    Dense(1, activation ="sigmoid"),
])

gan = Sequential([generator, discriminator])

In [8]:
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [9]:
batch_size=32
dataset=tf.data.Dataset.from_tensor_slices(X_train).shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

NameError: name 'tf' is not defined

In [10]:
def train_gan(gan, dataset, batch_size, codings_size, n_epochs=50):
    generator, discriminator = gan.layers
    for epoch in range(n_epochs):
        for X_batch in dataset:
            #phase 1 -train the disc
            noise = tf.random.normal(shape=[batch_size, codings_size])
            generated_images=generator(noise)
            X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
            y1 = tf.constant([[0.]]*batch_size + [[1.]]*batch_size)
            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)
            #phase 2 - train the gen
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]]*batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)
            
train_gan(gan, dataset, batch_size, codings_size)

NameError: name 'dataset' is not defined